<a href="https://colab.research.google.com/github/choiitea/Choi_DSPN_S24/blob/main/Exercise15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 15: Power analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below.

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill.

`c` is reading comprehension, and `x` is word reading experience.

In [1]:
sample_size = 100 # How many children in data set?
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0.
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa,
                          beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
                            # Generate ages distributed between age_lo and age_hi
                            ages <- runif(sample_size, min = age_lo, max = age_hi)
                            # Calculate word reading experience (x) based on age and add Gaussian noise
                            x <- beta_xa * ages + beta_x0 + rnorm(sample_size, mean = 0, sd = sd_x)
                            # Calculate reading comprehension (c) based on age, word reading experience, and add Gaussian noise
                            c <- beta_ca * ages + beta_cx * x + beta_c0 + rnorm(sample_size, mean = 0, sd = sd_c)
                            # Return a data frame containing age, word reading experience, and reading comprehension
                            return(data.frame(age = ages, word_reading_experience = x, reading_comprehension = c))
                          }

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,age,word_reading_experience,reading_comprehension
,<dbl>,<dbl>,<dbl>
1,116.2587,7.788229,171.1395
2,172.7556,124.824699,513.9073
3,176.6749,51.140622,157.8951
4,135.2671,43.255888,234.3591
5,100.8476,88.163574,334.0828
6,177.5615,75.574170,425.3733


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data.

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously.

In [4]:
# WRITE YOUR CODE HERE
#install.packages("mediation")
library(mediation)

run_analysis <- function(simulated_data) {
  fitM <- lm(word_reading_experience ~ age, data = simulated_data)

  fitY <- lm(reading_comprehension ~ age + word_reading_experience, data = simulated_data)

  # mediation
  fitMed <- mediate(fitM, fitY, treat = "age", mediator = "word_reading_experience")
  # vector containing ACME estimate, ACME pvalue, ADE estimate and ADE p-value
  acme_ade_pvalues <- c(fitMed$d0, fitMed$d0.p, fitMed$z0, fitMed$z0.p)

  return(acme_ade_pvalues)
  }

analysis_results <- run_analysis(dat)
print(analysis_results)

[1] 1.7805904 0.0000000 0.8975315 0.0040000


---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [21]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)

    # Start simulating
    for (i in 1:num_simulations) {
      # WRITE YOUR CODE HERE
      # simulate data
      simulated_data <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
      # Run analysis
      analysis_results <- run_analysis(simulated_data)
      # Store the outputs from each simulation in the simouts matrix
      simouts[i, ] <- analysis_results
      }
      # Calculate coverage for both ACME and ADE estimates using p-values in simouts
      ACME_cvg = mean(simouts[, 2] <= alpha)
      ADE_cvg =  mean(simouts[, 4] <= alpha)

    return(list(ACME_cvg = ACME_cvg, ADE_cvg = ADE_cvg))
}

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01.

In [22]:
# WRITE YOUR CODE HERE
num_simulations <- 10
alpha <- 0.01

sim10_results <- repeat_analysis(num_simulations, alpha, sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

print(sim10_results)


$ACME_cvg
[1] 0.6

$ADE_cvg
[1] 0.5



---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run.

In [23]:
# WRITE YOUR CODE HERE
library(tidyverse)
# parameters for testing sample sizes + 10 sim + alpha .01
sample_sizes <- c(50, 75, 100, 125, 150)
num_simulations <- 10
alpha <- 0.01

sample_size_sim10_results <- expand.grid(sample_size = sample_sizes, alpha = alpha)
sample_size_sim10_results$id <- 1:nrow(sample_size_sim10_results)

results <- sample_size_sim10_results %>%
  nest(parameters := c(sample_size, alpha)) %>%
    mutate(ACME_power = map(parameters, ~ repeat_analysis(num_simulations, .$alpha, .$sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)$ACME_cvg),
         ADE_power = map(parameters, ~ repeat_analysis(num_simulations, .$alpha, .$sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)$ADE_cvg)) %>%
      unnest(c(parameters, ACME_power, ADE_power))


Print your results.

In [24]:
# WRITE YOUR CODE HERE
print(results)

# A tibble: 5 × 5
     id sample_size alpha ACME_power ADE_power
  <int>       <dbl> <dbl>      <dbl>     <dbl>
1     1          50  0.01        0.5       0.4
2     2          75  0.01        0.8       0.6
3     3         100  0.01        1         0.7
4     4         125  0.01        0.9       0.8
5     5         150  0.01        1         1  


## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above?

> based on the results, we can generally see that increasing sample size improves the power to detect mediation effect and power to detect the direct effect (of course). In particular, sample size of 150 seems to be the best sample size to select (though it is depndent on the specific parameters we have selected). Selecting the exact power we want is somewhat arbitrary - for some 80% power may be sufficient in light of realistic constraints such as sample availability. But the output alone seems to suggest that 150 is sweetspot to approximately get 100% power with an alpha of .01.
>

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> The direct effects are harder to detect than the mediated effect because when analyzing the direct effect, part of the variance it explains in reading comprehension is already accounted for by the mediator (word reading experience).

**DUE:** 5pm EST, April 3, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*